In [1]:
import torch
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Reshape


/Users/chaewon/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rc('font', family='AppleGothic')

In [3]:
train = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/train.csv')
trade = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/international_trade.csv')
test = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/test.csv')

In [4]:
train = train.rename(columns={'supply(kg)': 'supply', 'price(원/kg)': 'price'})
train.drop(columns=['ID'], inplace=True) 
test.drop(columns=['ID'], inplace=True)

In [395]:
#연월일
train['year'] = pd.to_datetime(train['timestamp']).dt.year
train['month'] = pd.to_datetime(train['timestamp']).dt.month
train['day'] = pd.to_datetime(train['timestamp']).dt.day
train['ismarch'] = np.where(train['month'] == 3, 1, 0)


print(train.columns)


Index(['timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'ismarch'],
      dtype='object')


In [396]:
test['year'] = pd.to_datetime(test['timestamp']).dt.year
test['month'] = pd.to_datetime(test['timestamp']).dt.month
test['day'] = pd.to_datetime(test['timestamp']).dt.day
test['ismarch'] = np.where(test['month'] == 3, 1, 0)
print(test.columns)

Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'year', 'month',
       'day', 'ismarch'],
      dtype='object')


In [397]:
# 'price'와 'supply'가 0인 행을 해당 'item' 종목 및 'month' 별 평균값으로 대체
for index, row in train[(train['price'] == 0)].iterrows():
    item = row['item']
    month = row['month']
    
    # 해당 'item' 및 'month'에 대한 평균값 계산
    avg_price = train[(train['item'] == item) & (train['month'] == month)]['price'].mean()
    avg_supply = train[(train['item'] == item) & (train['month'] == month)]['supply'].mean()
    
    # 대체
    train.at[index, 'price'] = avg_price
    train.at[index, 'supply'] = avg_supply

# 변경된 'train' 데이터프레임 확인
print(train.head())


    timestamp item corporation location        supply        price  year  \
0  2019-01-01   TG           A        J  39655.713419  1588.269032  2019   
1  2019-01-02   TG           A        J  39681.297751  1589.293722  2019   
2  2019-01-03   TG           A        J  60601.000000  1728.000000  2019   
3  2019-01-04   TG           A        J  25000.000000  1408.000000  2019   
4  2019-01-05   TG           A        J  32352.000000  1250.000000  2019   

   month  day  ismarch  
0      1    1        0  
1      1    2        0  
2      1    3        0  
3      1    4        0  
4      1    5        0  


In [398]:
# 'train' 데이터프레임에 'month' 및 'item' 별 'supply' 평균 계산
mean_supply_by_month_item = train.groupby(['month', 'item'])['supply'].mean()

# 'test' 데이터프레임에 'month' 및 'item'을 기준으로 'supply' 평균값을 병합 (merge)
test_merged = pd.merge(test, mean_supply_by_month_item.reset_index(), on=['month', 'item'], how='left')

# 'supply_y' 열은 평균값을 의미하므로, 'supply_y' 열을 'supply'로 이름 변경
test_merged.rename(columns={'supply_y': 'supply'}, inplace=True)

# 'supply_x' 열은 병합 전 'test' 데이터프레임에 있던 열이므로 삭제
#test_merged.drop('supply_x', axis=1, inplace=True)

# 'test' 데이터프레임에 적용된 수정 사항을 확인
test = test_merged


In [399]:
df_encoded = pd.get_dummies(train, columns=['item', 'corporation', 'location'])
df_encoded.drop(columns=['timestamp'], inplace=True)
train= df_encoded

test.drop(columns=['timestamp'], inplace=True)
df_encoded2 = pd.get_dummies(test, columns=['item','corporation', 'location'])
test= df_encoded2


In [400]:
features = train.columns

In [401]:
train = train.sort_values(by=['year', 'month', 'day'])


In [402]:
X_train = train[['year', 'month', 'day','supply', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S','ismarch']]
y_train = train['price']


In [405]:
y_train = np.reshape(y_train, (len(y_train), 1))


In [406]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# 데이터 정규화
scaler_X = MinMaxScaler()
normalized_X_train = scaler_X.fit_transform(X_train)

scaler_y = MinMaxScaler()
normalized_y_train = scaler_y.fit_transform(y_train.reshape(-1, 1))

# LSTM에 입력으로 넣기 위한 형태로 변환
# (샘플 수, 타임 스텝 수, 특성 수)
# 여기서는 간단하게 타임 스텝을 1로 설정합니다.
X_train_reshaped = normalized_X_train.reshape((normalized_X_train.shape[0], 1, normalized_X_train.shape[1]))

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(X_train_reshaped, normalized_y_train, epochs=70, batch_size=32, shuffle=False)



Epoch 1/70
1857/1857 [==============================] - 2s 655us/step - loss: 0.0012
Epoch 2/70
1857/1857 [==============================] - 1s 592us/step - loss: 0.0014
Epoch 3/70
1857/1857 [==============================] - 1s 593us/step - loss: 0.0017
Epoch 4/70
1857/1857 [==============================] - 1s 590us/step - loss: 0.0019
Epoch 5/70
1857/1857 [==============================] - 1s 592us/step - loss: 0.0020
Epoch 6/70
1857/1857 [==============================] - 1s 591us/step - loss: 0.0020
Epoch 7/70
1857/1857 [==============================] - 1s 591us/step - loss: 0.0020
Epoch 8/70
1857/1857 [==============================] - 1s 587us/step - loss: 0.0020
Epoch 9/70
1857/1857 [==============================] - 1s 598us/step - loss: 0.0020
Epoch 10/70
1857/1857 [==============================] - 1s 586us/step - loss: 0.0019
Epoch 11/70
1857/1857 [==============================] - 1s 594us/step - loss: 0.0017
Epoch 12/70
1857/1857 [==============================] - 1s 589

In [409]:
# 새로운 데이터에 대한 예측
# (예측값을 다시 역정규화하여 원래 스케일로 되돌릴 수 있습니다.)
new_data = test
# 'train' 데이터프레임에서 사용한 feature들의 순서를 가져옴
train_feature_order = X_train.columns.tolist()

# 'new_data'에서 사용한 feature들의 순서를 가져옴
new_data_feature_order = new_data.columns.tolist()

# 'new_data'를 'train' 데이터프레임의 feature 순서에 맞게 재정렬
new_data_reordered = new_data[train_feature_order]

# 'scaler_X'를 다시 초기화하여 'new_data_reordered'를 변환
new_data_scaled = scaler_X.transform(new_data_reordered)
# 새로운 데이터도 정규화
new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

predicted_scaled = model.predict(new_data_reshaped)

# 예측 결과 역정규화
predicted = scaler_y.inverse_transform(predicted_scaled)

# 예측 결과 출력
print(predicted)

35/35 [==============================] - 0s 363us/step
[[3228.4966]
 [3313.3066]
 [3398.351 ]
 ...
 [1282.1215]
 [1300.9712]
 [1319.0067]]


In [410]:

submission = submission = pd.read_csv('/Users/chaewon/Desktop/공모전/DACON_제주/data/sample_submission.csv')
submission['answer'] = predicted
submission.to_csv('submission_lstm_pepero2.csv', index = False)
